In [1]:
import warnings
warnings.filterwarnings(action='once')

In [2]:
import dask
from dask.distributed import Client
dask.config.set({'temporary-directory': r'E:\dask-cache'})
client = Client()  # start distributed scheduler locally.
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.87 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59275,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.87 GiB
Comm: tcp://127.0.0.1:59297,Total threads: 3
Dashboard: http://127.0.0.1:59300/status,Memory: 3.97 GiB
Nanny: tcp://127.0.0.1:59278,


In [3]:
%%time
import dask.dataframe as dd
import glob
from typing import List, Dict

files = glob.glob(r'e:\temp\parquet\OKX-Books-*.parquet')

grouped_files: Dict[str, List[str]] = {}
for file in files:
    items = file.split('-')
    dsID = '-'.join(items[2:-2])
    
    if dsID in grouped_files:
        grouped_files[dsID].append(file)
    else:
        grouped_files[dsID] = [file]

CPU times: total: 922 ms
Wall time: 950 ms


In [10]:
import os
import dask
import pandas as pd

import pyarrow as pa 

schema = pa.schema([
    ('instId', pa.string()),
    ('price', pa.string()),
    ('size', pa.string()),
    ('numOrders', pa.int64()),
    ('side', pa.string()),
    ('timestamp', pa.int64()),
    ('prevSeqId', pa.int64()),
    ('seqId', pa.int64()),
    ('action', pa.string()),
    ('checksum', pa.int64())
])

root_path = r'E:\out3'
tasks = []
for dsID, files in grouped_files.items():
    out_path = os.path.join(root_path, dsID)
    
    tasks.append((out_path, files))

def compute_min_max_timestamp(partition):
    min_timestamp = partition['timestamp'].min()
    max_timestamp = partition['timestamp'].max()
    return pd.DataFrame({'min_timestamp': [min_timestamp], 'max_timestamp': [max_timestamp]})
    
def gen(task):
    out_path, files = task
    ddf = dd.read_parquet(files)
    
    # Repartition the dataframe
    ddf = ddf.repartition(partition_size='256MB')
    result = ddf.map_partitions(compute_min_max_timestamp, meta={'min_timestamp': 'int64', 'max_timestamp': 'int64'})
    return result

In [11]:
result = dask.compute(dask.delayed(gen)(tasks[0]))

In [42]:
print(type(result[0]))
print(result[0].npartitions)

# t = []
# for i in range(result[0].npartitions):
#     part = result[0].get_partition(i)
#     t.append((part.min_timestamp[0], part.max_timestamp[0]))
list(result[0].min_timestamp)

<class 'dask.dataframe.core.DataFrame'>
2


[1687054626603, 1689303680101]

In [45]:
import os
import dask
import pandas as pd

import pyarrow as pa 

schema = pa.schema([
    ('instId', pa.string()),
    ('price', pa.string()),
    ('size', pa.string()),
    ('numOrders', pa.int64()),
    ('side', pa.string()),
    ('timestamp', pa.int64()),
    ('prevSeqId', pa.int64()),
    ('seqId', pa.int64()),
    ('action', pa.string()),
    ('checksum', pa.int64())
])

root_path = r'E:\out3'
tasks = []
for dsID, files in grouped_files.items():
    out_path = os.path.join(root_path, dsID)
    
    tasks.append((out_path, files))

def compute_min_max_timestamp(partition):
    min_timestamp = partition['timestamp'].min()
    max_timestamp = partition['timestamp'].max()
    return pd.DataFrame({'min_timestamp': [min_timestamp], 'max_timestamp': [max_timestamp]})
    
def gen(task):
    out_path, files = task
    ddf = dd.read_parquet(files)
    
    
    # Repartition the dataframe
    ddf = ddf.repartition(partition_size='256MB')
    
    # get the name 
    result = ddf.map_partitions(compute_min_max_timestamp, meta={'min_timestamp': 'int64', 'max_timestamp': 'int64'})
    min_ts = list(result.min_timestamp)
    max_ts = list(result.max_timestamp)
    def name_func(partition_index):
        return f'part-{partition_index}-{min_ts[partition_index]}-{max_ts[partition_index]}.parquet'
    
    # Write the dataframe to parquet
    ddf.to_parquet(out_path, write_metadata_file=True, schema=schema, compression='gzip', name_function=name_func)

In [47]:
print(len(tasks))

859


In [46]:
%%time
jobs = []
for task in tasks:
    jobs.append(dask.delayed(gen)(task))

dask.compute(*jobs)


KeyError: 'data'

In [ ]:
import glob

dataset = {}
for files_path in glob.glob('e:\\out2\\*'):
    dataset[files_path.split('\\')[-1]] = files_path

In [ ]:
import bisect
import zlib
from numpy import int32
from pandas import DataFrame, Series
import ctypes
from typing import List, Tuple
import dask.dataframe as dd

class BooksItem:
    def __init__(self,
                price: str,
                size: str,
                numOrders: int,
                side: str,
                ) -> None:
        self.price = price
        self.size = size
        self.numOrders = numOrders
        self.side = side
    
    def __str__(self) -> str:
        # if self.size == '0':
        #     raise Exception('zero size')
        return ':'.join([self.price, self.size])

    def __lt__(self, other):
        return float(self.price) < float(other.price)
    

class Books:
    def __init__(self, instId: str) -> None:
        self.instId = instId
        self.bids: List[BooksItem] = []
        self.asks: List[BooksItem] = []
    
    def update(self, rawRecord: Series) -> None:
        if rawRecord['instId'] != self.instId:
            raise ValueError('instId mismatch')
        
        new_booksItem = BooksItem(
                            rawRecord['price'], 
                            rawRecord['size'], 
                            rawRecord['numOrders'], 
                            rawRecord['side']
                        )
        if rawRecord['side'] not in ['bid', 'ask']:
            raise ValueError('side should be either bid or ask')
        target = self.bids if rawRecord['side'] == 'bid' else self.asks
        if rawRecord['side'] == 'bid':
            # asks are sorted in descending order
            target.reverse()

        dest_level = next(filter(lambda x: x.price == new_booksItem.price, target), None)
        if dest_level is None:
            if new_booksItem.size == '0':
                # print(f'deprecated booksItem-> {new_booksItem.price}:{new_booksItem.size}')
                if rawRecord['side'] == 'bid':
                    target.reverse()
                return
            inserted_point = bisect.bisect_left(target, new_booksItem)
            target.insert(inserted_point, new_booksItem)
        else:
            if new_booksItem.size == '0':
                # print(f'{new_booksItem.price}: {new_booksItem.size}')
                target.remove(dest_level)
            else:
                dest_level.size = new_booksItem.size
                dest_level.numOrders = new_booksItem.numOrders

        if rawRecord['side'] == 'bid':
            # reverse back
            target.reverse()

    def update_many(self, ddf: DataFrame):
        if ddf.iloc[0]['action'] == 'snapshot':
            # print('Encounted snapshot action......')
            self.clear()

        for _, row in ddf.iterrows():
            self.update(row)

    def clear(self):
        self.bids = []
        self.asks = []
    
    def checksum_text(self) -> str:
        bids = self.bids[:25]
        asks = self.asks[:25]

        checksum_string = ''
        for i in range(max(len(bids), len(asks))):
            if i < len(bids):
                checksum_string = ':'.join([checksum_string, str(bids[i])])
            if i < len(asks):
                checksum_string = ':'.join([checksum_string, str(asks[i])])
        checksum_string = checksum_string.strip(':')
        return checksum_string
    
    @property
    def checksum(self) -> int:
        checksum_string = self.checksum_text()
        # print(f'checksum_string: {checksum_string}')
        # Calculate checksum with CRC32
        raw_checksum = ctypes.c_int32(zlib.crc32(checksum_string.encode('utf-8'))).value
        return raw_checksum


def verify(dataset_path: str) -> int:
    ddf = dd.read_parquet(dataset_path)
    ddf = ddf.compute()
    instId = ddf.head(1).iloc[0]['instId']
    books = Books(instId)
    
    print(f'{instId} -> Total num of rows: {len(ddf)})')
    
    passed_counter = 0
    failed_counter = 0
    current_checksum = None
    last_ts = 0
    for timestamp, sub_ddf in ddf.groupby('timestamp'):
        if timestamp - last_ts > 60*1000: # 数据中断
            if sub_ddf['action'].iloc[0] != 'snapshot':
                continue
            else:
                print(f'{instId} -> Restart at ts({timestamp})----------------<')
        
        last_ts = timestamp
        
        correct_checksum = sub_ddf['checksum'].iloc[0]
        books.update_many(sub_ddf)
        
        cal_checksum = books.checksum
        if cal_checksum != correct_checksum:
            failed_counter += 1
            # print((f"incorrect checksum at ts({timestamp}): {cal_checksum} != {correct_checksum}"))
            # print(f'Current: {passed_counter} / {failed_counter} : {100*passed_counter/(passed_counter+failed_counter)}%')
            # print(f'{instId} -> Current checksum_text: {books.checksum_text()}')
            # raise Exception(f"{instId} -> incorrect checksum at ts({timestamp}): {cal_checksum} != {correct_checksum}")
        else:
            passed_counter += 1
        if (passed_counter+failed_counter) % 100000 == 0:
            print(f'{instId} -> Current: {passed_counter} / {failed_counter} : {100*passed_counter/(passed_counter+failed_counter)}%')
    print(f'{instId} -> Result: {passed_counter} / {failed_counter} : {100*passed_counter/(passed_counter+failed_counter)}%')
    return failed_counter

当数据中断或下载不完整时, 校验会失败.

In [ ]:
%%time

verify(dataset['CRV-USDT-SWAP-400'])

# jobs = []
# for dsID in dataset:
#     jobs.append(dask.delayed(verify)(dataset[dsID]))

# result = dask.compute(*jobs)

In [ ]:
dsIDs = [i for i in dataset]
dsIDs